In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dad-dasd/my_fastai_model.pkl
/kaggle/input/sdasa-asdsa/Train/MountainSoil/LINE_ALBUM__250725_58.jpg
/kaggle/input/sdasa-asdsa/Train/MountainSoil/LINE_ALBUM__250725_37.jpg
/kaggle/input/sdasa-asdsa/Train/MountainSoil/LINE_ALBUM__250725_22.jpg
/kaggle/input/sdasa-asdsa/Train/MountainSoil/LINE_ALBUM__250725_92.jpg
/kaggle/input/sdasa-asdsa/Train/MountainSoil/LINE_ALBUM__250725_12.jpg
/kaggle/input/sdasa-asdsa/Train/MountainSoil/LINE_ALBUM__250725_31.jpg
/kaggle/input/sdasa-asdsa/Train/MountainSoil/LINE_ALBUM__250725_70.jpg
/kaggle/input/sdasa-asdsa/Train/MountainSoil/LINE_ALBUM__250725_44.jpg
/kaggle/input/sdasa-asdsa/Train/MountainSoil/LINE_ALBUM__250725_95.jpg
/kaggle/input/sdasa-asdsa/Train/MountainSoil/LINE_ALBUM__250725_64.jpg
/kaggle/input/sdasa-asdsa/Train/MountainSoil/LINE_ALBUM__250725_34.jpg
/kaggle/input/sdasa-asdsa/Train/MountainSoil/LINE_ALBUM__250725_39.jpg
/kaggle/input/sdasa-asdsa/Train/MountainSoil/LINE_ALBUM__250725_30.jpg
/kaggle/input/sdasa-asdsa/Train/Mo

In [2]:
from transformers import (
    RagTokenizer,
    RagRetriever,
    RagSequenceForGeneration,
    Trainer,
    TrainingArguments,
    RagTokenForGeneration,
    RagTokenizer,
    RagRetriever,
    RagSequenceForGeneration,
)
!pip install faiss-cpu
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import Dataset
import torch
import faiss
import numpy as np
import matplotlib.pyplot as plt
import os


2025-07-26 16:48:35.478843: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753548515.722892      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753548515.797697      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 53.1 MB/s eta 0:00:00


In [3]:
import gradio as gr
import os
from random import choice

# Dataset ฝังในโค้ด (ดินตะกอนน้ำ)
data = {
    "title": ["Alluvial Soil"] * 5,
    "text": [
        "ดินตะกอนน้ำเกิดจากการทับถมของตะกอนที่ถูกพัดพาโดยน้ำ มีองค์ประกอบของทราย ตะกอนละเอียด และดินเหนียว มักพบในบริเวณที่ราบลุ่มแม่น้ำ และสามเหลี่ยมปากแม่น้ำ",
        "ดินตะกอนน้ำมีความอุดมสมบูรณ์สูง เพราะมีแร่ธาตุและสารอาหารที่จำเป็นต่อพืช เช่น ไนโตรเจน ฟอสฟอรัส และโพแทสเซียม",
        "ข้อเสียของดินตะกอนน้ำ ได้แก่ การระบายน้ำไม่ดี ความเสี่ยงต่อน้ำท่วม ความอุดมสมบูรณ์ต่ำในบางกรณี และการกัดเซาะ",
        "ดินตะกอนน้ำเหมาะสำหรับปลูกพืช เช่น ข้าว ข้าวโพด อ้อย มันสำปะหลัง พืชผักต่างๆ พืชไร่ และไม้ผลบางชนิด",
        "ดินตะกอนน้ำสามารถช่วยกรองตะกอนและสารปนเปื้อนจากน้ำ ทำให้คุณภาพน้ำดีขึ้น"
    ],
    "question": [
        "ดินตะกอนน้ำเกิดขึ้นได้อย่างไร?",
        "ดินตะกอนน้ำมีแร่ธาตุสำคัญอะไรบ้าง?",
        "ข้อเสียของดินตะกอนน้ำมีอะไรบ้าง?",
        "พืชชนิดใดปลูกได้ดีในดินตะกอนน้ำ?",
        "ดินตะกอนน้ำช่วยในเรื่องคุณภาพน้ำอย่างไร?"
    ],
    "answers": [
        {
            "text": ["เกิดจากการทับถมของตะกอนที่ถูกพัดพาโดยน้ำ"],
            "answer_start": [15]
        },
        {
            "text": ["ไนโตรเจน ฟอสฟอรัส และโพแทสเซียม"],
            "answer_start": [84]
        },
        {
            "text": ["การระบายน้ำไม่ดี ความเสี่ยงต่อน้ำท่วม ความอุดมสมบูรณ์ต่ำในบางกรณี และการกัดเซาะ"],
            "answer_start": [28]
        },
        {
            "text": ["ข้าว ข้าวโพด อ้อย มันสำปะหลัง พืชผักต่างๆ พืชไร่ และไม้ผล"],
            "answer_start": [40]
        },
        {
            "text": ["ช่วยกรองตะกอนและสารปนเปื้อนจากน้ำ ทำให้คุณภาพน้ำดีขึ้น"],
            "answer_start": [15]
        }
    ]
}

def qa_from_dataset(user_question):
    for i, question in enumerate(data["question"]):
        if user_question.strip() == question.strip():
            context = data["text"][i]
            answer = data["answers"][i]["text"][0]

            # เปลี่ยนเป็นโฟลเดอร์รูปภาพใหม่
            image_folder = "/kaggle/input/sdasa-asdsa/Train"
            
            all_images = []
            if os.path.exists(image_folder):
                # รับไฟล์ภาพทุกไฟล์ .jpg .jpeg .png
                all_images = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
            
            if all_images:
                img_path = choice(all_images)
            else:
                img_path = None

            return f"📖 บริบท: {context}\n✅ คำตอบ: {answer}", img_path
    return "❌ ไม่พบคำถามนี้ในฐานข้อมูล", None

with gr.Blocks(title="ถาม-ตอบเกี่ยวกับดินตะกอนน้ำ (Step 1)") as demo:
    gr.Markdown("# 🌱 ระบบถาม-ตอบ: ดินตะกอนน้ำ (Alluvial Soil)")
    gr.Markdown("กรุณาเลือกหรือพิมพ์คำถามเกี่ยวกับดินตะกอนน้ำ เพื่อดูคำตอบจาก dataset และภาพประกอบ")

    with gr.Row():
        dropdown = gr.Dropdown(choices=data["question"], label="📝 คำถาม", interactive=True)
        textbox = gr.Textbox(label="หรือพิมพ์คำถามเอง", placeholder="ลองพิมพ์: ดินตะกอนน้ำมีแร่ธาตุสำคัญอะไรบ้าง?")

    answer_box = gr.Textbox(label="คำตอบ", lines=4)
    soil_image = gr.Image(label="ภาพประกอบดินตะกอนน้ำ")

    def handle_input(selected, typed):
        question = typed.strip() if typed else selected
        return qa_from_dataset(question)

    submit = gr.Button("🔍 หาคำตอบ")
    submit.click(fn=handle_input, inputs=[dropdown, textbox], outputs=[answer_box, soil_image])

demo.launch()


* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://b9ac02dcdad2027cb6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
